In [1]:
import pandas as pd
import json
import numpy as np
import operator
import matplotlib.pyplot as plt
from collections import Counter
import os
import time
import pickle

ROOT = os.path.dirname(os.getcwd())

In [2]:
events = pd.read_pickle(os.path.join(ROOT, 'processed_data', 'events_df.pkl.gz'))
display(events)

eventId           subEventName  \
Italy 0             8            Simple pass   
      1             8            Simple pass   
      2             7                  Touch   
      3             1  Ground attacking duel   
      4             1  Ground attacking duel   
...               ...                    ...   
Spain 628654        8            Simple pass   
      628655        7           Acceleration   
      628656        8                  Cross   
      628657        8            Simple pass   
      628658        7              Clearance   

                                                  tags  playerId  \
Italy 0                                 [{'id': 1801}]      8327   
      1                                 [{'id': 1801}]     20438   
      2                                             []      8306   
      3       [{'id': 504}, {'id': 703}, {'id': 1801}]      8306   
      4       [{'id': 503}, {'id': 703}, {'id': 1801}]      8306   
...                                                ...       ...   
Spain 628654                            [{'id': 1801}]     20623   
      628655                            [{'id': 1801}]    122832   
      628656               [{'id': 401}, {'id': 1802}]    122832   
      628657                            [{'id': 1801}]     40756   
      628658              [{'id': 1401}, {'id': 1802}]      3306   

                                              positions  matchId  \
Italy 0        [{'y': 52, 'x': 49}, {'y': 44, 'x': 43}]  2575959   
      1        [{'y': 44, 'x': 43}, {'y': 17, 'x': 36}]  2575959   
      2        [{'y': 17, 'x': 36}, {'y': 56, 'x': 78}]  2575959   
      3        [{'y': 56, 'x': 78}, {'y': 15, 'x': 64}]  2575959   
      4        [{'y': 15, 'x': 64}, {'y': 15, 'x': 72}]  2575959   
...                                                 ...      ...   
Spain 628654    [{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]  2565927   
      628655    [{'y': 2, 'x': 88}, {'y': 21, 'x': 97}]  2565927   
      628656   [{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]  2565927   
      628657     [{'y': 74, 'x': 8}, {'y': 56, 'x': 9}]  2565927   
      628658  [{'y': 56, 'x': 9}, {'y': 100, 'x': 100}]  2565927   

                       eventName  teamId matchPeriod     eventSec subEventId  \
Italy 0                     Pass    3158          1H     2.530536         85   
      1                     Pass    3158          1H     3.768418         85   
      2       Others on the ball    3158          1H     4.868265         72   
      3                     Duel    3158          1H     8.114676         11   
      4                     Duel    3158          1H     8.647892         11   
...                          ...     ...         ...          ...        ...   
Spain 628654                Pass     682          2H  2939.077491         85   
      628655  Others on the ball     682          2H  2940.515560         70   
      628656                Pass     682          2H  2942.098761         80   
      628657                Pass     675          2H  2943.089232         85   
      628658  Others on the ball     675          2H  2943.668846         71   

                     id  
Italy 0       180423957  
      1       180423958  
      2       180423959  
      3       180423960  
      4       180423961  
...                 ...  
Spain 628654  253302671  
      628655  253302673  
      628656  253302674  
      628657  253302698  
      628658  253302695  

[3071395 rows x 12 columns]

In [20]:
attempts = events.loc[events['eventName'] == 'Shot'].groupby(['matchId', 'teamId']).count().mean(axis=1)

In [25]:
goals = events.loc[events['eventName'] == 'Shot']
goals['goals'] = (goals['tags'].apply(lambda tags: any([tag in [{'id': 101}] for tag in tags]))).astype(int)
goals = goals.loc[goals['goals']==1].groupby(['matchId', 'teamId']).count().mean(axis=1)
goals = goals.reindex(attempts.index).fillna(0)



<ipython-input-25-af9f1fcaa783>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  goals['goals'] = (goals['tags'].apply(lambda tags: any([tag in [{'id': 101}] for tag in tags]))).astype(int)


In [31]:
df = pd.concat([goals, attempts], keys=['goals', 'attempts'], axis=1)
df

goals  attempts
matchId teamId                 
2499719 1609      4.0      27.0
        1631      3.0       7.0
2499720 1625      1.0      12.0
        1651      0.0       6.0
2499721 1610      2.0      15.0
...               ...       ...
2576336 3315      0.0      14.0
2576337 3164      1.0       9.0
        3204      2.0      14.0
2576338 3185      2.0       4.0
        3193      1.0      11.0

[3649 rows x 2 columns]

In [73]:
for row in df.iterrows():
    match, team = row[0]
    try: 
        opponent = [e for e in df.loc[match].index if e != team][0]
    except:
        goals_team, attempts_team = row[1]['goals'], row[1]['attempts']
        df.loc[row[0], 'pezzali'] = 0
        conversion_rate = goals_team/attempts_team
        df.loc[row[0], 'conversion_rate'] = conversion_rate
        continue
    
    goals_team, attempts_team = row[1]['goals'], row[1]['attempts']
    
    opponent_data = df.loc[(match, opponent)]
    goals_opponent, attempts_opponent = opponent_data['goals'], opponent_data['attempts']
    
    pezzali = goals_team/attempts_team * attempts_opponent/goals_opponent
    conversion_rate = goals_team/attempts_team
    
    df.loc[row[0], 'pezzali'] = pezzali
    df.loc[row[0], 'conversion_rate'] = conversion_rate
    
df = df.fillna(np.inf)
df = df.replace([np.inf, -np.inf], np.nan)

<ipython-input-73-6a4cf0284974>:17: RuntimeWarning: divide by zero encountered in double_scalars
  pezzali = goals_team/attempts_team * attempts_opponent/goals_opponent
<ipython-input-73-6a4cf0284974>:17: RuntimeWarning: invalid value encountered in double_scalars
  pezzali = goals_team/attempts_team * attempts_opponent/goals_opponent


In [74]:
df.to_pickle(os.path.join(ROOT, 'intermediates', 'pezzali.pkl.gz'))

In [75]:
df

goals  attempts   pezzali  conversion_rate
matchId teamId                                            
2499719 1609      4.0      27.0  0.345679         0.148148
        1631      3.0       7.0  2.892857         0.428571
2499720 1625      1.0      12.0       NaN         0.083333
        1651      0.0       6.0  0.000000         0.000000
2499721 1610      2.0      15.0  0.400000         0.133333
...               ...       ...       ...              ...
2576336 3315      0.0      14.0       NaN         0.000000
2576337 3164      1.0       9.0  0.777778         0.111111
        3204      2.0      14.0  1.285714         0.142857
2576338 3185      2.0       4.0  5.500000         0.500000
        3193      1.0      11.0  0.181818         0.090909

[3649 rows x 4 columns]